## Project Lab Notebook
### Marina Herrera Sarrias


## Date: 2018-12-31

I started by defining the project's layout following the recommended structure by the course centerpiece gideline. I also created a github repository (__[https://github.com/msarrias/protein_multial_noise_reduction](https://github.com/msarrias/protein_multial_noise_reduction)__
) to keep track of the project's versions. I downloaded the fixed data sets that will be used for the experiment and defined a list of tasks and things I would like to include and take into consideration following the guidelines. 

Project Layout:
Top level organization: protein_multial_noise_reduction<br>

./doc/ <br>
./data/<br>
./src/<br>
./bin/<br>
./results/<br>



./data/ <br>
$\qquad$./data/raw_data/<br>
$\qquad$./data/noise_reduced_data/<br>


As the whole project is performed only with the data sets contained in appbio11, I wont follow a chronological organization for the data directory. (adviced on the __[Course centerpice](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000424#s8)__) but I will rename the directory to *raw_data*.


the **doc**  recository will contain the project_report, the **data** repository will contain the raw_data, the **src** directory will contain all of my scrips, I am still now very sure if I will use the **bin** repository, finally the **results** directory will contain the labnotebook,  a set of test data and a set of control cases files used to verify the program's error handling.

**raw_data:** containes six subdirectories:

./raw_data/ <br>
$\qquad$./raw_data/asymmetric_0.5/<br>
$\qquad$./raw_data/asymmetric_1.0/<br>
$\qquad$./raw_data/asymmetric_2.0/<br>
$\qquad$./raw_data/symmetric_0.5/<br>
$\qquad$./raw_data/symmetric_1.0/<br>
$\qquad$./raw_data/symmetric_2.0/<br>



### Date: 2019-01-01

Project setup: I defined which structure I will like to follow to develop the project. And tried to understand what is the project's idea and what is it that I want to achive with it. I defined the major tasks I have to develope and how I want to structure the scripts. I will divide the program's tasks in three parts, which are:
1. Filtering and redusing noisy columns.
2. Caluculating distance matrix and inferring the trees.
3. Comparing the trees calculating the symmetric distance.<br>
After coding the different parts of the program I will introduce the error handling and test the program with a reduced data set of the raw_data, which can be found on the results directory.<br>

Find and Remove Noisy colums:I defined a function (noise_filter) for filtering the noisy columns. Aterwards I created a function (reduce_noise) for reducing the noise in the alignments, and finally a function (perform_noise_reduction) for reading the file and performing the reduction. The program geenrates a new directory for the noise reduced alignments.

### Date: 2019-01-02

I created the script for generating the distance matrices and the inferred trees (for the original and noise reduced tree). For this task I used the phylogenetic software FastPhylo.The program Fastprot was used for generating the distance matrices and the program fnj for inferring the trees. The  function computing_and_writing_alignment_tree performs both tasks. A file with the inferred tree is finally saved on both (raw_data and noise_reduced_data) directories. <br>
<br>
For the last task of the project, using the Python package DendroPy. I computed the symmetric distances between the reference trees and inferred trees (both cases, noise reduced and original alignments) with the function compare_trees, and the function compute_distance_between_trees for reading the tree and performing the computation.

### Date: 2019-01-03

I created a data set for testing the three scripts, this data set was taken from the original raw_data set, I also introduced some sequences I created myself for controlling the functionality of the programs. I introduced error handling at all levels of the scripts.

I generated a final scripts that performs all the tasks of the project.

### Date: 2019-01-04

I runned the run_program script  on the fixed data and started working on the statistics.

### Date: 2019-01-05

Generated the project report using Latex and added comments to the script.